Test Gain Calculation

_________
Imports

In [ ]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal

import scipy.constants as const

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data')
meta_file_name = 'wfs_meta.dat'
ccd_name = 'ccd_pump_off.txt'
wfs_name = 'wfs_pump_off.txt'
meta_path_to_file = os.path.join(package_data_dir, meta_file_name)
ccd_path_to_file = os.path.join(package_data_dir, ccd_name)
wfs_path_to_file = os.path.join(package_data_dir, wfs_name)

files = PKDict(
    meta = meta_path_to_file,
    ccd = ccd_path_to_file,
    wfs = wfs_path_to_file
    )

_____________
Total number of excited states in full crystal, as a function of the number of crystal slices

In [ ]:
params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = 50,
        pulseE      = 1.0e-6,
        tau_fwhm    = 300.0e-12 / math.sqrt(2.),
        sigx_waist  = 1.2e-3 /1.18,
        sigy_waist  = 1.2e-3 /1.18,
)

num_slices = 6
crystal_params = PKDict(
    length = 2.5/100.0,  # [m]
    nslice = 6,
    n0          = [1.76 for _ in range(num_slices)],
    n2          = [16.0 for _ in range(num_slices)],
    pump_energy = 0.035,  # [J]
    pump_waist  = 1.2e-3 /1.18,  # [m]
)

e_crystal = crystal.Crystal(crystal_params)

prop_type = 'gain_calc'

max_num_c_slices = 10

total_n_excited_states_i = np.zeros(max_num_c_slices)
total_n_excited_states   = np.zeros(max_num_c_slices)

for num_c_slices in np.arange(max_num_c_slices):

    num_slices = num_c_slices+1
    crystal_params = PKDict(
        length = 2.5/100.0,  # [m]
        nslice = num_c_slices+1,
        n0          = [1.76 for _ in range(num_slices)],
        n2          = [16.0 for _ in range(num_slices)],
        pump_energy = 0.035,  # [J]
        pump_waist  = 1.2e-3 /1.18,  # [m]
    )
    e_crystal = crystal.Crystal(crystal_params)
    nslices_crystal = len(e_crystal.slice)
    thisPulse = pulse.LaserPulse(params)#, files)
    nslices_pulse = len(thisPulse.slice)
    
    dx = (e_crystal.slice[0].pop_inversion_xfin-e_crystal.slice[0].pop_inversion_xstart)/e_crystal.slice[0].pop_inversion_nx
    dy = (e_crystal.slice[0].pop_inversion_yfin-e_crystal.slice[0].pop_inversion_ystart)/e_crystal.slice[0].pop_inversion_ny
    cell_volume = dx*dy*e_crystal.slice[0].length
    
    for crystal_index_i in np.arange(nslices_crystal):
        total_n_excited_states_i[num_c_slices] += np.sum(np.sum(e_crystal.slice[crystal_index_i].pop_inversion_mesh * cell_volume))
    
    if (num_c_slices < max_num_c_slices):
        print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for crystal_index_p in np.arange(nslices_crystal):
        total_n_excited_states[num_c_slices] += np.sum(np.sum(e_crystal.slice[crystal_index_p].pop_inversion_mesh * cell_volume))

n_c_slice = np.arange(1,max_num_c_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_c_slice,total_n_excited_states_i,'k',label='after initialization')
plt.plot(n_c_slice,total_n_excited_states,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Number Excited States')
ax.set_xlabel(r'Number of Crystal Slices')


________
Number of excited states as a function of longitudinal distance through the crystal

In [ ]:
params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = 50,
        pulseE      = 1.0e-6,
        tau_fwhm    = 300.0e-12 / math.sqrt(2.),
        sigx_waist  = 1.2e-3 /1.18,
        sigy_waist  = 1.2e-3 /1.18,
)
prop_type = 'gain_calc'

num_slices = 10
crystal_params = PKDict(
    length = 2.5/100.0,  # [m]
    nslice = 10,
    n0          = [1.76 for _ in range(num_slices)],
    n2          = [16.0 for _ in range(num_slices)],
    pump_energy = 0.035,  # [J]
    pump_waist  = 1.2e-3 /1.18,  # [m]
)
e_crystal = crystal.Crystal(crystal_params)
nslices_crystal = len(e_crystal.slice)

dx = (e_crystal.slice[0].pop_inversion_xfin-e_crystal.slice[0].pop_inversion_xstart)/e_crystal.slice[0].pop_inversion_nx
dy = (e_crystal.slice[0].pop_inversion_yfin-e_crystal.slice[0].pop_inversion_ystart)/e_crystal.slice[0].pop_inversion_ny
cell_volume = dx*dy*e_crystal.slice[0].length

thisPulse = pulse.LaserPulse(params)#, files)
nslices_pulse = len(thisPulse.slice)

n_excited_states_slice_i = np.zeros(nslices_crystal)
n_excited_states_slice = np.zeros(nslices_crystal)

for c_slice_index in np.arange(nslices_crystal):
    n_excited_states_slice_i[c_slice_index] = np.sum(np.sum(e_crystal.slice[c_slice_index].pop_inversion_mesh * cell_volume))

print('\nPropagating ', nslices_pulse, 'laser slices through ', num_c_slices+1,' crystal slices')
thisPulse = e_crystal.propagate(thisPulse, prop_type)

for c_slice_index in np.arange(nslices_crystal):
    n_excited_states_slice[c_slice_index] = np.sum(np.sum(e_crystal.slice[c_slice_index].pop_inversion_mesh * cell_volume))

#Assume all crystals have the same length
z_c_slice = e_crystal.slice[0].length *(np.arange(nslices_crystal)+0.5)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(z_c_slice,n_excited_states_slice_i,'k',label='after initialization')
plt.plot(z_c_slice,n_excited_states_slice,'--k',label='after propagation')
#plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Number Excited States')
ax.set_xlabel(r'Distance from Crystal Face [m]')
ax.set_title('    Longitudinal Distribution Excited States')


___________
Total number of photons in full laser pulse, as a function of the number of laser slices

In [ ]:
num_slices = 5
crystal_params = PKDict(
    length = 2.5/100.0,  # [m]
    nslice = 5,
    n0          = [1.76 for _ in range(num_slices)],
    n2          = [16.0 for _ in range(num_slices)],
    pump_energy = 0.035,  # [J]
    pump_waist  = 1.2e-3 /1.18,  # [m]
)
prop_type = 'gain_calc'

max_num_l_slices = 10

total_n_photons_i = np.zeros(max_num_l_slices)
total_n_photons   = np.zeros(max_num_l_slices)

for num_l_slices in np.arange(max_num_l_slices):
    
    params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = num_l_slices+1,
        pulseE      = 1.0e-6,
        tau_fwhm    = 300.0e-12 / math.sqrt(2.),
        sigx_waist  = 1.2e-3 /1.18,
        sigy_waist  = 1.2e-3 /1.18,
    )
    
    e_crystal = crystal.Crystal(crystal_params)
    thisPulse = pulse.LaserPulse(params)#, files)
    nslices_pulse = len(thisPulse.slice)
    nslices_crystal = len(e_crystal.slice)
    
    for laser_index_i in np.arange(nslices_pulse):
        total_n_photons_i[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh))
    
    if (num_l_slices < max_num_l_slices):
        print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for laser_index_p in np.arange(nslices_pulse):
        total_n_photons[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh))

n_l_slice = np.arange(1,max_num_l_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_l_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

_________
Single pass, number of photons in laser as a function of number of slices in crystal

In [ ]:
params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = 50,
        pulseE      = 1.0e-6,
        tau_fwhm    = 300.0e-12 / math.sqrt(2.),
        sigx_waist  = 1.2e-3 /1.18,
        sigy_waist  = 1.2e-3 /1.18,
)
prop_type = 'gain_calc'

max_num_c_slices = 10

total_n_photons_i = np.zeros(max_num_c_slices)
total_n_photons   = np.zeros(max_num_c_slices)

for num_c_slices in np.arange(max_num_c_slices):
    
    num_slices = num_c_slices+1
    crystal_params = PKDict(
        length = 2.5/100.0,  # [m]
        nslice = num_c_slices+1,
        n0          = [1.76 for _ in range(num_slices)],
        n2          = [16.0 for _ in range(num_slices)],
        pump_energy = 0.035,  # [J]
        pump_waist  = 1.2e-3 /1.18,  # [m]
    )
    e_crystal = crystal.Crystal(crystal_params)
    thisPulse = pulse.LaserPulse(params)#,files)
    nslices_pulse = len(thisPulse.slice)
    
    for laser_index_i in np.arange(nslices_pulse):
        total_n_photons_i[num_c_slices] += np.sum(np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh))
    
    if (num_c_slices < max_num_c_slices):
        print('\nPropagating ', nslices_pulse, 'laser slices through ', num_c_slices+1,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for laser_index_p in np.arange(nslices_pulse):
        total_n_photons[num_c_slices] += np.sum(np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh))

n_c_slice = np.arange(1,max_num_c_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_c_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_c_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Crystal Slices')
